In [ ]:
# a. How often does the candidate who raised more money win a race?
# b. How often does the candidate who spent more money win a race?
# c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?
# d. How often does the incumbent candidate win a race?
# e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?

In [15]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
# read in dataset

hor = pd.read_csv(r"/Users/mauriaparker/Documents/NSS_Projects/Python/webscraping-open-secrets-scary-data-fest/data/HOR_2020.csv")
hor

,Candidate,Party,State,District,Incumbent,Winner,Percentage of Vote,Raised,Spent,Funds Remaining
0,Jerry Carl,(R),Alabama,District 01,False,True,64.9,1971321,1859349,111973
1,James Averhart,(D),Alabama,District 01,False,False,35.0,80095,78973,1122
2,Barry Moore,(R),Alabama,District 02,False,True,65.3,650807,669368,-13633
3,Phyllis Harvey-Hall,(D),Alabama,District 02,False,False,34.6,56050,55988,0
4,Mike D Rogers,(R),Alabama,District 03,True,True,67.5,1193111,1218564,502234
...,...,...,...,...,...,...,...,...,...,...
877,Tricia Zunker,(D),Wisconsin,District 07,False,False,39.2,1261957,1232690,29267
878,Mike Gallagher,(R),Wisconsin,District 08,True,True,64.0,3202905,2841801,1169362
879,Amanda Stuck,(D),Wisconsin,District 08,False,False,36.0,416978,399916,2165
880,Liz Cheney,(R),Wyoming,District 01,True,True,68.6,3003883,3060167,153567


In [ ]:
# a) How often does the candidate who raised more money win a race?
# while state & district are the same & raised = max, check correlation with winner = true?
    # Tomo: get max per state & district, filter by winner = true

In [6]:
# rank by raised & create new column
hor['RaisedRank'] = hor.groupby(['State', 'District'])['Raised'].rank(ascending=False)
hor

,Candidate,Party,State,District,Incumbent,Winner,Percentage of Vote,Raised,Spent,Funds Remaining,RaisedRank
0,Jerry Carl,(R),Alabama,District 01,False,True,64.9,1971321,1859349,111973,1.0
1,James Averhart,(D),Alabama,District 01,False,False,35.0,80095,78973,1122,2.0
2,Barry Moore,(R),Alabama,District 02,False,True,65.3,650807,669368,-13633,1.0
3,Phyllis Harvey-Hall,(D),Alabama,District 02,False,False,34.6,56050,55988,0,2.0
4,Mike D Rogers,(R),Alabama,District 03,True,True,67.5,1193111,1218564,502234,1.0
...,...,...,...,...,...,...,...,...,...,...,...
877,Tricia Zunker,(D),Wisconsin,District 07,False,False,39.2,1261957,1232690,29267,2.0
878,Mike Gallagher,(R),Wisconsin,District 08,True,True,64.0,3202905,2841801,1169362,1.0
879,Amanda Stuck,(D),Wisconsin,District 08,False,False,36.0,416978,399916,2165,2.0
880,Liz Cheney,(R),Wyoming,District 01,True,True,68.6,3003883,3060167,153567,1.0


In [19]:
# create column making rank categorical across ranks(most or not)
hor['Top Raised'] = np.where(hor['RaisedRank'] == 1.0, True, False)
hor

,Candidate,Party,State,District,Incumbent,Winner,Percentage of Vote,Raised,Spent,Funds Remaining,RaisedRank,Top Raised
0,Jerry Carl,(R),Alabama,District 01,False,True,64.9,1971321,1859349,111973,1.0,True
1,James Averhart,(D),Alabama,District 01,False,False,35.0,80095,78973,1122,2.0,False
2,Barry Moore,(R),Alabama,District 02,False,True,65.3,650807,669368,-13633,1.0,True
3,Phyllis Harvey-Hall,(D),Alabama,District 02,False,False,34.6,56050,55988,0,2.0,False
4,Mike D Rogers,(R),Alabama,District 03,True,True,67.5,1193111,1218564,502234,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
877,Tricia Zunker,(D),Wisconsin,District 07,False,False,39.2,1261957,1232690,29267,2.0,False
878,Mike Gallagher,(R),Wisconsin,District 08,True,True,64.0,3202905,2841801,1169362,1.0,True
879,Amanda Stuck,(D),Wisconsin,District 08,False,False,36.0,416978,399916,2165,2.0,False
880,Liz Cheney,(R),Wyoming,District 01,True,True,68.6,3003883,3060167,153567,1.0,True


In [21]:
# Cross tabulation between Top Raised and Winner 

CrosstabResult = pd.crosstab(index=hor['Top Raised'], columns= hor['Winner'])
print(CrosstabResult)

# chisq of Top Raised & Winner
from scipy.stats import chi2_contingency
chisqrankwin = chi2_contingency(CrosstabResult)
chisqrankwin

Winner      False  True 
Top Raised              
False         402     47
True           51    382


Chi2ContingencyResult(statistic=530.3409977933018, pvalue=2.381019513073205e-117, dof=1, expected_freq=array([[230.60884354, 218.39115646],
       [222.39115646, 210.60884354]]))

In [23]:
# percentage of time raising the most money results in winning

382/433

0.8822170900692841